In [98]:
import os
import json
import re
import pprint as p
import pandas as pd
from utils import drop_last_pair, return_original_dict, extract_labels_preds, exact_match_metrics, relax_match_metrics, return_max_sim_text,\
sectag_to_regex, find_segs, ros_seg

# # Add argument for filename and model input
# import argparse
# parser = argparse.ArgumentParser(description="file")
# parser.add_argument("--filename", type=str, required=True, default='sample_393')
# args = parser.parse_args()

# model_folder = "Output_llama3.1_8b"
# model_folder = "Output_mistral-small_24b"
# model_folder = "Output_gemma3_27b"
# model_folder = 'Output_gpt-oss_20b'

# model_folder = "Output_llama3.1_8b_no_sim"
model_folder = "Output_mistral-small_24b_no_sim"
# model_folder = "Output_gemma3_27b_no_sim"
# model_folder = 'Output_gpt-oss_20b_no_sim'


%run utils.py

# Performance Evaluation

In [104]:
filename = []
tp_l_em = []
fp_l_em = []
od_l_em = []
ud_l_em = []
tp_l_rm = []
fp_l_rm = []
od_l_rm = []
ud_l_rm = []
for args_filename in sorted(os.listdir(model_folder)):
# Read the label and prediction files
    with open(f"input/{args_filename}/label_extract.json",'r') as f:
        label_converted = json.load(f)

    with open(f"/home/hieunghiem/CHSI/ROS_sentence_transformers/scaling/{model_folder}/{args_filename}/output_extract.json",'r') as file:
        pred_converted = json.load(file)

    print(f"RESULT FOR FILE {args_filename}")

    ## Extract label and prediction for EM, RM, OD, UD
    exact_matched_labels = []
    exact_matched_preds = []

    relax_matched_labels = []
    relax_matched_preds = []  

    over_detection = []
    under_detection = []

    label_list_trim = drop_last_pair(label_converted)
    pred_list_trim = drop_last_pair(pred_converted)

    for label in label_list_trim:
        for pred in pred_list_trim:
            # Find overlapping keys
            # print(set(pred.keys()).difference('status'))
            common_keys = set(pred.keys()) & set(label.keys())

            if common_keys:
                #Check if it's exact match (all keys match)
                if set(pred.keys()) == set(label.keys()):
                    exact_matched_labels.append(label)
                    exact_matched_preds.append(pred)

                else:
                    relax_matched_labels.append(label)
                    relax_matched_preds.append(pred)

    #over/under detection
    for pred in pred_list_trim:
        if pred not in exact_matched_labels and pred not in relax_matched_labels and pred not in exact_matched_preds and pred not in relax_matched_preds:

                over_detection.append(pred)

    # print(over_detection)         
    for label in label_list_trim:
        if label not in exact_matched_preds and label not in relax_matched_preds and label not in exact_matched_labels and label not in relax_matched_labels:
                under_detection.append(label)
         
    #mapping back with the original
    exact_matched_labels_final = return_original_dict(label_converted, exact_matched_labels)
    exact_matched_preds_final = return_original_dict(pred_converted, exact_matched_preds)
    relax_matched_labels_final = return_original_dict(label_converted, relax_matched_labels)
    relax_matched_preds_final = return_original_dict(pred_converted, relax_matched_preds)

    over_detection_final = return_original_dict(pred_converted, over_detection)
    under_detection_final = return_original_dict(label_converted, under_detection)

    
    print(f"Over detection: {over_detection_final}")  
    precision_em, sensitivity_em, f1_em, tp_em, fp_em, ud_em, od_em = exact_match_metrics(exact_matched_labels_final, exact_matched_preds_final,
                        under_detection_final, relax_matched_preds_final, over_detection_final)

    precision_rm, sensitivity_rm, f1_rm, tp_rm, fp_rm, ud_rm, od_rm = relax_match_metrics(exact_matched_labels_final, exact_matched_preds_final,
                        under_detection_final, relax_matched_labels_final, relax_matched_preds_final, over_detection_final)
    
    filename.append(args_filename)
    tp_l_em.append(tp_em)
    fp_l_em.append(fp_em)
    od_l_em.append(od_em)
    ud_l_em.append(ud_em)

    tp_l_rm.append(tp_rm)
    fp_l_rm.append(fp_rm)
    od_l_rm.append(od_rm)
    ud_l_rm.append(ud_rm)



RESULT FOR FILE sample_1248
Over detection: []
[]
em:1, fp:0, ud:0,od: 0
1.0 1.0 1.0
[]
rm:1, fp:0, ud:0,od: 0
1.0 1.0 1.0
RESULT FOR FILE sample_1252
Over detection: []
[]
em:9, fp:0, ud:0,od: 1
0.9 1.0 0.95
[]
rm:10, fp:0, ud:0,od: 0
1.0 1.0 1.0
RESULT FOR FILE sample_1419
Over detection: []
[]
em:4, fp:0, ud:1,od: 4
0.5 0.8 0.62
[]
rm:8, fp:0, ud:1,od: 0
1.0 0.89 0.94
RESULT FOR FILE sample_1495
Over detection: []
[]
em:6, fp:0, ud:1,od: 1
0.86 0.86 0.86
[]
rm:7, fp:0, ud:1,od: 0
1.0 0.88 0.94
RESULT FOR FILE sample_1505
Over detection: [{'6': 'ENT', 'status': 'negative'}]
[]
em:13, fp:0, ud:0,od: 1
0.93 1.0 0.96
[]
rm:13, fp:0, ud:0,od: 1
0.93 1.0 0.96
RESULT FOR FILE sample_1592
Over detection: [{'unknown_entity': 'over_detection', 'status': 'positive'}]
[]
em:18, fp:0, ud:2,od: 2
0.9 0.9 0.9
[{'label': {'46': 'muscle', '47': 'weakness', 'status': 'positive'},
  'pred': {'47': 'weakness', 'status': 'negative'}}]
rm:18, fp:1, ud:2,od: 1
0.9 0.9 0.9
RESULT FOR FILE sample_1921
Over 

In [105]:
od_l_em

[0, 1, 4, 1, 1, 2, 0, 1, 3, 5, 71, 2, 5, 5, 3, 3, 3, 0, 9, 4, 1, 0, 0, 0]

In [106]:
print(f"EM: {sum(tp_l_em)}, FP: {sum(fp_l_em)}, UD: {sum(ud_l_em)}, OD: {sum(od_l_em)}")

print(f"RM: {sum(tp_l_rm)}, FP: {sum(fp_l_rm)}, UD: {sum(ud_l_rm)}, OD: {sum(od_l_rm)}")

EM: 248, FP: 12, UD: 47, OD: 124
RM: 284, FP: 14, UD: 47, OD: 86


In [102]:
precision_em = sum(tp_l_em)/(sum(tp_l_em)+sum(fp_l_em)+sum(od_l_em))
sensitivity_em = sum(tp_l_em)/(sum(tp_l_em)+sum(ud_l_em))
f1_em = 2 * precision_em * sensitivity_em / (precision_em + sensitivity_em)

print(f"Precision EM: {precision_em:.4f}")
print(f"Sensitivity EM: {sensitivity_em:.4f}")
print(f"f1 score EM: {f1_em:.4f}")

Precision EM: 0.5547
Sensitivity EM: 0.8192
f1 score EM: 0.6615


In [103]:
precision_rm = sum(tp_l_rm)/(sum(tp_l_rm)+sum(fp_l_rm)+sum(od_l_rm))
sensitivity_rm = sum(tp_l_rm)/(sum(tp_l_rm)+sum(ud_l_rm))
f1_rm = 2 * precision_rm * sensitivity_rm / (precision_rm + sensitivity_rm)

print(f"Precision RM: {precision_rm:.4f}")
print(f"Sensitivity RM: {sensitivity_rm:.4f}")
print(f"f1 score RM: {f1_rm:.4f}")

Precision RM: 0.6198
Sensitivity RM: 0.8351
f1 score RM: 0.7115
